## 1. Setup

In [ ]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '..', 'published_model_main'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

from deepsequence.seasonal_component import SeasonalComponent
from deepsequence.tabnet_encoder import TabNetEncoder
from deepsequence.cross_layer import CrossNetwork
from deepsequence.unit_norm import UnitNorm

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("✓ Setup complete!")

## 2. Generate Intermittent Demand Data

Creating synthetic data with 80% zeros (intermittent demand pattern).

In [ ]:
n_skus, n_weeks = 10, 100
dates = pd.date_range('2020-01-01', periods=n_weeks, freq='W')

data_list = []
for sku_id in range(n_skus):
    for i, date in enumerate(dates):
        # 80% zeros (intermittent)
        if np.random.random() < 0.8:
            qty = 0
        else:
            seasonal = 10 * np.sin(2 * np.pi * i / 52)
            trend = 0.1 * i
            qty = max(0, seasonal + trend + np.random.randint(5, 20))
        
        data_list.append({
            'ds': date, 'id_var': sku_id, 'Quantity': qty,
            'week_of_year': date.isocalendar()[1],
            'month': date.month, 'day_of_week': date.dayofweek
        })

data = pd.DataFrame(data_list)
print(f"Dataset: {len(data):,} records, Sparsity: {(data['Quantity']==0).sum()/len(data)*100:.1f}%")
data.head()

## 3. Feature Engineering

In [ ]:
data = data.sort_values(['id_var', 'ds']).reset_index(drop=True)

# Lags
for lag in [1, 2, 4]:
    data[f'lag_{lag}'] = data.groupby('id_var')['Quantity'].shift(lag)
data[['lag_1', 'lag_2', 'lag_4']] = data[['lag_1', 'lag_2', 'lag_4']].fillna(0)

# Fourier features
data['fourier_weekly_sin'] = np.sin(2 * np.pi * data.index / 52)
data['fourier_weekly_cos'] = np.cos(2 * np.pi * data.index / 52)
data['fourier_monthly_sin'] = np.sin(2 * np.pi * data.index / 12)
data['fourier_monthly_cos'] = np.cos(2 * np.pi * data.index / 12)

print(f"✓ Features: {len(data.columns)} columns")

## 4. Train/Test Split (80/20 by time per SKU)

In [ ]:
train_list, test_list = [], []
for sku_id in data['id_var'].unique():
    sku = data[data['id_var'] == sku_id]
    split = int(len(sku) * 0.8)
    train_list.append(sku.iloc[:split])
    test_list.append(sku.iloc[split:])

train = pd.concat(train_list, ignore_index=True)
test = pd.concat(test_list, ignore_index=True)

print(f"Train: {len(train):,}, Test: {len(test):,}")
print(f"Test sparsity: {(test['Quantity']==0).sum()/len(test)*100:.1f}%")

## 5. Build DeepSequence Model

All 4 components + enhancements + intermittent handler.

In [ ]:
n_ids = train['id_var'].nunique()

# Shared ID embedding
id_in = layers.Input(shape=(1,), name='id')
id_emb = layers.Flatten()(layers.Embedding(n_ids + 1, 16)(id_in))

# 1. SEASONAL
seasonal_comp = SeasonalComponent(train, ['Quantity'], 'id_var', 4, True, True, 'w')
seasonal_comp.seasonal_feature()
s_cols = [c for c in seasonal_comp.sr_df.columns if c not in ['id_var', 'ds']]

s_ins, s_embs = [], []
for col in s_cols:
    s_i = layers.Input(shape=(1,), name=f's_{col}')
    s_ins.append(s_i)
    if col in ['week_of_year', 'month']:
        nu = 52 if col == 'week_of_year' else 12
        s_embs.append(layers.Flatten()(layers.Embedding(nu+1, 8)(s_i)))
    else:
        s_embs.append(s_i)

f_ins = [layers.Input(shape=(1,), name=f) for f in ['fourier_weekly_sin', 'fourier_weekly_cos', 
                                                      'fourier_monthly_sin', 'fourier_monthly_cos']]

s_cat = layers.Concatenate()([id_emb] + s_embs + f_ins)
s_tab = TabNetEncoder(32, 32, 2, 2, 2)(s_cat)
s_cr = CrossNetwork(2)(s_tab)
s_norm = UnitNorm()(s_cr)
s_out = layers.Dense(1, activation='linear', name='s_forecast')(s_norm)

# 2. TREND
t_in = layers.Input(shape=(1,), name='time')
t_cat = layers.Concatenate()([id_emb, t_in])
t_h = layers.Dropout(0.2)(layers.Dense(32, activation='relu')(t_cat))
t_out = layers.Dense(1, activation='linear', name='t_forecast')(t_h)

# 3. REGRESSOR
l_ins = [layers.Input(shape=(1,), name=f'lag_{i}') for i in [1,2,4]]
r_cat = layers.Concatenate()([id_emb] + l_ins)
r_tab = TabNetEncoder(16, 16, 2, 1, 1)(r_cat)
r_cr = CrossNetwork(1)(r_tab)
r_norm = UnitNorm()(r_cr)
r_out = layers.Dense(1, activation='linear', name='r_forecast')(r_norm)

# 4. HOLIDAY
h_in = layers.Input(shape=(1,), name='holiday')
h_cat = layers.Concatenate()([id_emb, h_in])
h_out = layers.Dense(1, activation='linear', name='h_forecast')(layers.Dense(16, 'relu')(h_cat))

# COMBINE
combined = layers.Add()([s_out, t_out, r_out, h_out])

# CROSS-COMPONENT
cc = CrossNetwork(2)(layers.Concatenate()([s_out, t_out, r_out, h_out]))

# INTERMITTENT HANDLER
ih = layers.Dropout(0.2)(UnitNorm()(layers.Dense(32, 'relu')(cc)))
ih = UnitNorm()(layers.Dense(16, 'relu')(ih))
prob = layers.Dense(1, activation='sigmoid', name='prob')(ih)

# FINAL
final = layers.Multiply(name='forecast')([combined, prob])

model = Model([id_in] + s_ins + f_ins + [t_in] + l_ins + [h_in], final)
print(f"✓ Model: {model.count_params():,} params")

## 6. Prepare Data

In [ ]:
def prep(df, sc, s_cols):
    sc.data = df
    sc.seasonal_feature()
    s_df = sc.sr_df
    
    ids = df['id_var'].values[:len(s_df)].reshape(-1, 1)
    s_vals = [s_df[c].values.reshape(-1, 1) for c in s_cols]
    f_vals = [df[f].values[:len(s_df)].reshape(-1, 1) 
              for f in ['fourier_weekly_sin', 'fourier_weekly_cos', 
                       'fourier_monthly_sin', 'fourier_monthly_cos']]
    t_val = ((df['ds'] - df['ds'].min()).dt.days.values[:len(s_df)] / 
             (df['ds'].max() - df['ds'].min()).days).reshape(-1, 1)
    l_vals = [df[f'lag_{i}'].values[:len(s_df)].reshape(-1, 1) for i in [1,2,4]]
    h_val = (df['month'].values[:len(s_df)] == 12).astype(float).reshape(-1, 1)
    
    return [ids] + s_vals + f_vals + [t_val] + l_vals + [h_val], df['Quantity'].values[:len(s_df)]

train_X, train_y = prep(train, seasonal_comp, s_cols)
test_X, test_y = prep(test, seasonal_comp, s_cols)

print(f"✓ Train: {len(train_y):,}, Test: {len(test_y):,}")

## 7. Train

In [ ]:
model.compile(optimizer='adam', loss='mae', metrics=['mae'])

history = model.fit(
    train_X, train_y, validation_split=0.2,
    epochs=30, batch_size=32, verbose=1,
    callbacks=[EarlyStopping('val_loss', patience=5, restore_best_weights=True)]
)

print(f"\n✓ Trained ({len(history.history['loss'])} epochs)")

## 8. Evaluate

In [ ]:
test_pred = np.maximum(model.predict(test_X, verbose=0).flatten(), 0)
test_mae = mean_absolute_error(test_y, test_pred)
test_rmse = np.sqrt(mean_squared_error(test_y, test_pred))

print(f"Test MAE:  {test_mae:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Val')
plt.title('Training')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.scatter(test_y, test_pred, alpha=0.5)
plt.plot([0, test_y.max()], [0, test_y.max()], 'r--')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Predictions')
plt.grid(True)
plt.tight_layout()
plt.show()

## 9. Intermittent Handler Analysis

In [ ]:
prob_model = Model(model.inputs, model.get_layer('prob').output)
test_prob = prob_model.predict(test_X, verbose=0).flatten()

is_zero = (test_y == 0).astype(int)
print(f"Avg prob when actual=0: {test_prob[is_zero==1].mean():.3f}")
print(f"Avg prob when actual>0: {test_prob[is_zero==0].mean():.3f}")

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.hist(test_prob[is_zero==1], bins=20, alpha=0.7, label='Zero')
plt.hist(test_prob[is_zero==0], bins=20, alpha=0.7, label='Non-Zero')
plt.xlabel('Probability')
plt.ylabel('Count')
plt.title('Intermittent Handler')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.scatter(test_prob, test_y, alpha=0.3)
plt.xlabel('Probability')
plt.ylabel('Actual')
plt.title('Prob vs Demand')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 10. Sample Forecast

In [ ]:
test_df = test.copy()
test_df['forecast'] = test_pred[:len(test)]

sku = test_df['id_var'].unique()[0]
sku_all = data[data['id_var'] == sku].sort_values('ds')
sku_test = test_df[test_df['id_var'] == sku].sort_values('ds')

plt.figure(figsize=(12, 5))
plt.plot(sku_all['ds'], sku_all['Quantity'], label='Historical', linewidth=2)
plt.plot(sku_test['ds'], sku_test['forecast'], label='Forecast', 
         color='red', linewidth=2, marker='o')
plt.axvline(train['ds'].max(), color='gray', linestyle='--', label='Split')
plt.title(f'SKU {sku}', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Quantity')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\n✓ Demo complete!")

## Summary

**Demonstrated:**
- 4-component architecture (Seasonal, Trend, Regressor, Holiday)
- Fourier features for smooth seasonality
- TabNet + CrossNetwork for feature selection/interactions
- Intermittent handler for zero demand prediction
- Full pipeline: data → model → evaluation

**Key Results:**
- Synthetic data with 80% sparsity
- Model: ~67K parameters
- Intermittent handler learns zero vs non-zero patterns

**Production Results (Real Data):**
- MAE: 3.1941 vs LightGBM: 3.2499 (1.7% improvement)
- 86.4% sparse, 10 SKUs
- See research_comparison.log for details

**Next Steps:**
- Replace with your own dataset
- Tune hyperparameters
- Add more features
- Compare with baselines

See `README.md` for full documentation.